In [4]:
import unittest
from unittest.mock import patch, MagicMock
import requests

In [5]:
# ------------------------
# 1 - Função is_par
# ------------------------
def is_par(n: int) -> bool:
    return n % 2 == 0

class TestIsPar(unittest.TestCase):
    def test_numeros(self):
        self.assertTrue(is_par(2))     # par
        self.assertFalse(is_par(3))    # ímpar
        self.assertTrue(is_par(0))     # zero
        self.assertTrue(is_par(-4))    # negativo par
        self.assertFalse(is_par(-3))   # negativo ímpar

In [6]:
# ------------------------
# 2 - Função fatorial
# ------------------------
def fatorial(n: int) -> int:
    if n < 0:
        raise ValueError("n deve ser >= 0")
    if n == 0:
        return 1
    resultado = 1
    for i in range(1, n + 1):
        resultado *= i
    return resultado

class TestFatorial(unittest.TestCase):
    def test_fatorial(self):
        self.assertEqual(fatorial(0), 1)      # caso base
        self.assertEqual(fatorial(5), 120)    # valor conhecido
        with self.assertRaises(ValueError):   # erro esperado
            fatorial(-1)

In [7]:
# ------------------------
# 3 - Classe Conta
# ------------------------
class InsufficientFunds(Exception):
    pass

class Conta:
    def __init__(self, saldo=0):
        self.saldo = saldo

    def depositar(self, amount):
        if amount < 0:
            raise ValueError("Depósito inválido")
        self.saldo += amount

    def sacar(self, amount):
        if amount < 0:
            raise ValueError("Saque inválido")
        if amount > self.saldo:
            raise InsufficientFunds("Saldo insuficiente")
        self.saldo -= amount

class TestConta(unittest.TestCase):
    def test_conta(self):
        conta = Conta(100)
        conta.depositar(50)
        self.assertEqual(conta.saldo, 150)  # depósito ok

        conta.sacar(30)
        self.assertEqual(conta.saldo, 120)  # saque ok

        with self.assertRaises(InsufficientFunds): # saldo insuficiente
            conta.sacar(500)

        with self.assertRaises(ValueError): # depósito inválido
            conta.depositar(-10)

        with self.assertRaises(ValueError): # saque inválido
            conta.sacar(-5)

In [8]:

# ------------------------
# 4 - Função buscar_clima (mock requests)
# ------------------------
def buscar_clima(cidade: str):
    try:
        response = requests.get(f"https://api.exemplo/clima?cidade={cidade}")
        data = response.json()
        if "temperatura" not in data:
            raise KeyError("Resposta sem temperatura")
        return data["temperatura"]
    except Exception as e:
        return f"Erro ao buscar clima: {e}"

class TestBuscarClima(unittest.TestCase):

    @patch("requests.get")
    def test_sucesso(self, mock_get):
        mock_response = MagicMock()
        mock_response.json.return_value = {"temperatura": 25}
        mock_get.return_value = mock_response

        self.assertEqual(buscar_clima("São Paulo"), 25)

    @patch("requests.get")
    def test_sem_temperatura(self, mock_get):
        mock_response = MagicMock()
        mock_response.json.return_value = {"umidade": 80}
        mock_get.return_value = mock_response

        resultado = buscar_clima("São Paulo")
        self.assertIn("Erro", resultado)

    @patch("requests.get")
    def test_excecao(self, mock_get):
        mock_get.side_effect = Exception("Falha na API")

        resultado = buscar_clima("São Paulo")
        self.assertIn("Erro", resultado)

In [9]:
# ------------------------
# Rodar todos os testes
# ------------------------
if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

............
----------------------------------------------------------------------
Ran 12 tests in 0.017s

OK
